In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dfc = pd.read_excel("/content/drive/My Drive/PythonP/online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = dfc.copy()

In [ ]:
# Variables: Değişkenler
# InvoiceNo (If it starts with C, the transaction is canceled): Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice (in sterling): Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [ ]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [ ]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [ ]:
df = df[~df["Invoice"].str.contains("C", na=False)]
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,515255.00000,10.95663,104.35401,-9600.00000,1.00000,3.00000,10.00000,19152.00000
Price,515255.00000,3.95637,127.68856,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [ ]:
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)
df["TotalPrice"] = df["Quantity"] * df["Price"]
cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})
cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

In [ ]:
cltv_c

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000
...,...,...,...
18283.00000,6,336,641.77000
18284.00000,1,494,461.68000
18285.00000,1,145,427.00000


In [ ]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
cltv_c[["average_order_value"]]

,average_order_value
Customer ID,
12346.00000,33.89636
12347.00000,661.66000
12348.00000,222.16000
12349.00000,890.38000
12351.00000,300.93000
...,...
18283.00000,106.96167
18284.00000,461.68000
18285.00000,427.00000


In [ ]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
cltv_c[["purchase_frequency"]]

,purchase_frequency
Customer ID,
12346.00000,0.00255
12347.00000,0.00046
12348.00000,0.00023
12349.00000,0.00070
12351.00000,0.00023
...,...
18283.00000,0.00139
18284.00000,0.00023
18285.00000,0.00023


In [ ]:
repeat_rate = cltv_c[(cltv_c["total_transaction"] > 1)].shape[0] / cltv_c.shape[0]
churn_rate = 1 - repeat_rate
print("repeat_rate:", repeat_rate)
print("churn_rate:", churn_rate)

repeat_rate: 0.6706073249884098
churn_rate: 0.3293926750115902


In [ ]:
cltv_c['profit_margin'] = cltv_c['total_price'] * 0.10
cltv_c[['profit_margin']]

,profit_margin
Customer ID,
12346.00000,37.28600
12347.00000,132.33200
12348.00000,22.21600
12349.00000,267.11400
12351.00000,30.09300
...,...
18283.00000,64.17700
18284.00000,46.16800
18285.00000,42.70000


In [ ]:
cltv_c['customer_value'] = cltv_c['average_order_value'] * cltv_c["purchase_frequency"]
cltv_c[['customer_value']]

,customer_value
Customer ID,
12346.00000,0.08643
12347.00000,0.30675
12348.00000,0.05150
12349.00000,0.61918
12351.00000,0.06976
...,...
18283.00000,0.14876
18284.00000,0.10702
18285.00000,0.09898


In [ ]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


In [ ]:
cltv_c.sort_values(by="cltv", ascending=False).tail()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061
14103.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000
14827.00000,1,5,0.00000,0.00000,0.00023,0.00000,0.00000,0.00000


In [ ]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276,A
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829,A
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615,A
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978,A
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003,A


In [ ]:
clv =cltv_c.groupby("segment").agg({"count", "mean", "sum"})
print(clv)

        total_transaction              total_unit                total_price  \
                     mean    sum count       mean      sum count        mean   
segment                                                                        
D                 1.22892   1326  1079  109.00463   117616  1079   178.18826   
C                 2.00371   2160  1078  283.05659   305135  1078   475.89652   
B                 3.76902   4063  1078  680.15863   733211  1078  1131.35918   
A                10.81186  11666  1079 4062.33735  4383262  1079  6401.40546   

                            average_order_value                     \
                  sum count                mean          sum count   
segment                                                              
D        192265.13000  1079           157.37300 169805.46550  1079   
C        513016.45300  1078           294.45487 317422.35514  1078   
B       1219605.20000  1078           389.72832 420127.12365  1078   
A       6907116.491

In [ ]:
def create_cltv_c(dataframe, profit=0.10):

    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[(dataframe['Quantity'] > 0)]
    dataframe.dropna(inplace=True)
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                                   'Quantity': lambda x: x.sum(),
                                                   'TotalPrice': lambda x: x.sum()})
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_c

In [ ]:
cltv_c.to_csv("/content/drive/My Drive/PythonP/cltc_c.csv")